Notebook to analyse raw waypoint data for Berlin.   
One can find that there are several trips recorded from the same devive. It would be interesting to find those trips and see how often they occur.


In [1]:
import sys, os
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import skmob as skm
# as jupyter notebook cannot find __file__, import module and submodule path via current_folder
current_folder = globals()['_dh'][0]

PROJECT_SRC_PATH = os.path.realpath(os.path.join(current_folder, '..', '..', 'xml4uf'))
sys.path.append(PROJECT_SRC_PATH)

from ufo_map.Utils.helpers import *

In [7]:
path = '/Users/Felix/cluster_remote/p/projects/vwproject/S3 Unzipped Felix/Berlin/1_Jan-31_Dec_2017/data/waypoints/waypoints.csv'
path_trips = '/Users/Felix/cluster_remote/p/projects/vwproject/S3 Unzipped Felix/Berlin/1_Jan-31_Dec_2017/data/trips/trips.csv'

In [11]:
chunks = pd.read_csv(path, chunksize=int(1E3),header=None)
for idx,df in enumerate(chunks):
    print('reading in sample of size {}'.format(len(df)))
    if idx==0: break

reading in sample of size 1000


In [12]:
# waypint content
df

,0,1,2,3,4,5,6,7,8,9,10
0,00459212658da1dfcf8f92a49cebd9db,0,2017-03-27T07:57:17.000Z,52.500600,13.209500,NaN,NaN,NaN,2b1c0b728274657cffa70af7b93ad9ca,59.0,kph
1,00459212658da1dfcf8f92a49cebd9db,1,2017-03-27T08:00:24.000Z,52.509900,13.231500,NaN,NaN,NaN,2b1c0b728274657cffa70af7b93ad9ca,22.0,kph
2,0067bbb2a9b38fb02f80d975655cf393,0,2017-03-27T17:33:11.113Z,52.484100,13.320700,NaN,NaN,NaN,89c43459f52692c5419f22e63a7f223c,NaN,kph
3,0067bbb2a9b38fb02f80d975655cf393,1,2017-03-27T17:36:15.417Z,52.477181,13.306794,NaN,NaN,NaN,89c43459f52692c5419f22e63a7f223c,NaN,kph
4,0067bbb2a9b38fb02f80d975655cf393,2,2017-03-27T17:39:18.953Z,52.473311,13.289916,NaN,NaN,NaN,89c43459f52692c5419f22e63a7f223c,NaN,kph
...,...,...,...,...,...,...,...,...,...,...,...
995,0932bf0ace176bc6b53e9ae45bf34b3c,8,2017-03-27T07:34:52.000Z,52.453335,13.438337,NaN,NaN,NaN,5e9ac912b66df274bd5d17ce02f7bdcc,16.0,kph
996,0932bf0ace176bc6b53e9ae45bf34b3c,9,2017-03-27T07:36:25.000Z,52.460190,13.435292,NaN,NaN,NaN,5e9ac912b66df274bd5d17ce02f7bdcc,32.0,kph
997,0932bf0ace176bc6b53e9ae45bf34b3c,10,2017-03-27T07:38:32.000Z,52.462280,13.434705,NaN,NaN,NaN,5e9ac912b66df274bd5d17ce02f7bdcc,6.0,kph
998,0932bf0ace176bc6b53e9ae45bf34b3c,11,2017-03-27T07:41:25.000Z,52.464500,13.431400,NaN,NaN,NaN,5e9ac912b66df274bd5d17ce02f7bdcc,4.0,kph


In [13]:
# trip point content
chunks = pd.read_csv(path_trips, chunksize=int(1E3),header=None)
for idx,df in enumerate(chunks):
    print('reading in sample of size {}'.format(len(df)))
    if idx==0: break

reading in sample of size 1000


In [14]:
df_head = pd.read_csv('/Users/Felix/cluster_remote/p/projects/vwproject/S3 Unzipped Felix/Berlin/1_Jan-31_Dec_2017/schema/TripBulkReportTripsHeaders.csv')
df_head

,TripId,DeviceId,ProviderId,Mode,StartDate,StartWDay,EndDate,EndWDay,StartLocLat,StartLocLon,...,DestinationZoneName,MultipleZones,MultipleCorridors,EndpointType,TripMeanSpeedKph,TripMaxSpeedKph,TripDistanceMeters,MovementType,OriginCbg,DestCbg


In [17]:
# assign columns
df.columns = df_head.columns
df.columns

Index(['TripId', 'DeviceId', 'ProviderId', 'Mode', 'StartDate', 'StartWDay',
       'EndDate', 'EndWDay', 'StartLocLat', 'StartLocLon', 'EndLocLat',
       'EndLocLon', 'IsStartHome', 'IsEndHome', 'GeospatialType',
       'ProviderType', 'ProviderDrivingProfile', 'VehicleWeightClass',
       'ProbeSourceType', 'OriginZoneName', 'DestinationZoneName',
       'MultipleZones', 'MultipleCorridors', 'EndpointType',
       'TripMeanSpeedKph', 'TripMaxSpeedKph', 'TripDistanceMeters',
       'MovementType', 'OriginCbg', 'DestCbg'],
      dtype='object')

In [25]:
df.loc[df.DeviceId.duplicated(keep=False)]

,TripId,DeviceId,ProviderId,Mode,StartDate,StartWDay,EndDate,EndWDay,StartLocLat,StartLocLon,...,DestinationZoneName,MultipleZones,MultipleCorridors,EndpointType,TripMeanSpeedKph,TripMaxSpeedKph,TripDistanceMeters,MovementType,OriginCbg,DestCbg
179,1daf9ea8c82fd532d8368458ff251753,cae8072aaf978457bcef17c10800d5cd,0e65972dce68dad4d52d063967f0a705,1,2017-02-10T15:46:37.000Z,5,2017-02-10T16:57:29.000Z,5,52.2278,11.7568,...,Berlin,True,NaN,1,105.986060,167.640970,125181.312974,1,NaN,NaN
333,381bfd824bdd6eaa2eb7c7ee81341ef4,7533e38b7971f5e07637738af5ca7656,2a79ea27c279e471f4d180b08d62b00a,1,2017-02-10T18:33:41.000Z,5,2017-02-10T19:04:18.000Z,5,52.5540,13.2923,...,NaN,True,NaN,3,64.723364,111.728991,33026.894502,1,NaN,NaN
565,617240f60d0baedeb0015d0f58dc85b9,cae8072aaf978457bcef17c10800d5cd,0e65972dce68dad4d52d063967f0a705,1,2017-02-11T04:22:26.000Z,6,2017-02-11T04:33:32.000Z,6,52.5142,13.4196,...,Berlin,False,NaN,0,8.669489,60.987604,1603.855532,1,NaN,NaN
747,828617ef295afd8c1d5ecacd9612623f,7533e38b7971f5e07637738af5ca7656,2a79ea27c279e471f4d180b08d62b00a,1,2017-02-11T15:46:54.000Z,6,2017-02-11T16:37:33.000Z,6,52.3996,13.0478,...,Berlin,True,NaN,3,22.541569,62.924370,19028.841320,1,NaN,NaN


In [27]:
df.loc[df.DeviceId == '7533e38b7971f5e07637738af5ca7656']

,TripId,DeviceId,ProviderId,Mode,StartDate,StartWDay,EndDate,EndWDay,StartLocLat,StartLocLon,...,DestinationZoneName,MultipleZones,MultipleCorridors,EndpointType,TripMeanSpeedKph,TripMaxSpeedKph,TripDistanceMeters,MovementType,OriginCbg,DestCbg
333,381bfd824bdd6eaa2eb7c7ee81341ef4,7533e38b7971f5e07637738af5ca7656,2a79ea27c279e471f4d180b08d62b00a,1,2017-02-10T18:33:41.000Z,5,2017-02-10T19:04:18.000Z,5,52.5540,13.2923,...,NaN,True,NaN,3,64.723364,111.728991,33026.894502,1,NaN,NaN
747,828617ef295afd8c1d5ecacd9612623f,7533e38b7971f5e07637738af5ca7656,2a79ea27c279e471f4d180b08d62b00a,1,2017-02-11T15:46:54.000Z,6,2017-02-11T16:37:33.000Z,6,52.3996,13.0478,...,Berlin,True,NaN,3,22.541569,62.924370,19028.841320,1,NaN,NaN


## create file to analyse concatenate all 14 files

In [30]:
# analysis
print('Number of unique trips {} and devices {} '.format(len(set(df.TripId)),len(set(df.DeviceId))))

Number of unique trips 1000 and devices 998 


In [32]:
# get freq of occurence
df['freq'] = df.groupby('DeviceId')['DeviceId'].transform('count')
#take only top x%
perc = 0.10
df.sort_values('freq',ascending=False).head(int(len(df)*perc)).to_csv(os.path.join(path_out,'trips_top_'+str(n_sample)+'p_device_occ.csv'),index=False))

,TripId,DeviceId,ProviderId,Mode,StartDate,StartWDay,EndDate,EndWDay,StartLocLat,StartLocLon,...,MultipleCorridors,EndpointType,TripMeanSpeedKph,TripMaxSpeedKph,TripDistanceMeters,MovementType,OriginCbg,DestCbg,count,freq
0,000aeeffd2410ec4c5bad7ffde115407,186bcc2c6710b7dc19367c858f276ee2,52720e003547c70561bf5e03b95aa99f,1,2017-02-10T12:15:19.463Z,5,2017-02-10T12:24:35.723Z,5,52.5056,13.5582,...,NaN,0,21.225044,31.739980,3279.622973,1,NaN,NaN,NaN,1
1,001956bb5bf92645eed9e579e757e104,6883133e74a88ab1549ff560155c240a,94f6d7e04a4d452035300f18b984988c,1,2017-02-11T10:08:06.000Z,6,2017-02-11T10:28:13.000Z,6,52.5684,13.5724,...,NaN,0,32.839842,82.633552,11010.469272,1,NaN,NaN,NaN,1
2,00207ed74a87340cbe3f64c7b1901482,ac1d32033fb42695c9b7c54b350619a4,0a09c8844ba8f0936c20bd791130d6b6,1,2017-02-11T06:32:21.000Z,6,2017-02-11T06:47:41.000Z,6,52.5113,13.3138,...,NaN,0,22.875698,55.839310,5846.011727,1,NaN,NaN,NaN,1
3,002762034a67a7e28966939a58b0a37b,53adeba4314554ad52e4ce4e2d1a9bfd,42a0e188f5033bc65bf8d78622277c4e,1,2017-02-09T11:31:44.000Z,4,2017-02-09T12:06:15.000Z,4,52.5343,13.1957,...,NaN,0,24.994492,58.012451,14378.775593,1,NaN,NaN,NaN,1
4,002e9ab96e0fa7d23e052061348e1719,d4ef9de0b7e0f5dc5b41f097730fb17f,52720e003547c70561bf5e03b95aa99f,1,2017-02-11T15:06:19.860Z,6,2017-02-11T15:15:24.533Z,6,52.5181,13.1965,...,NaN,0,24.318142,41.088581,3679.287658,1,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,ae82060a1093b3a52a19b3bfe0208a37,fb4cbbf91d27a611c52caf1feac4242f,00ec53c4682d36f5c4359f4ae7bd7ba1,1,2017-02-09T15:35:11.000Z,4,2017-02-09T15:36:26.000Z,4,52.4934,13.3173,...,NaN,1,17.534935,19.758809,365.311146,1,NaN,NaN,NaN,1
996,ae8eb419098322042d84f55d146ad205,3bc981308e8c4561367722d896e6389d,52720e003547c70561bf5e03b95aa99f,1,2017-02-10T20:02:39.600Z,5,2017-02-10T20:08:44.470Z,5,52.5106,13.3069,...,NaN,0,42.398486,59.402562,4297.204347,1,NaN,NaN,NaN,1
997,aea72f316d58c5f76e19cca478b8c3fa,58c98f0c4efd8ff8d157c5d6f5cb733d,94f6d7e04a4d452035300f18b984988c,1,2017-02-11T09:17:13.000Z,6,2017-02-11T09:48:47.000Z,6,52.5182,13.4307,...,NaN,0,26.641178,60.623907,14016.219871,1,NaN,NaN,NaN,1
998,aeb1645924c76e8d5957841542b43ea9,5d08655d13edaa3fdfb67cd62697432b,94f6d7e04a4d452035300f18b984988c,1,2017-02-09T11:44:32.000Z,4,2017-02-09T12:09:29.000Z,4,52.5182,13.1964,...,NaN,0,64.732410,112.694024,26917.893876,1,NaN,NaN,NaN,1


,TripId,DeviceId,ProviderId,Mode,StartDate,StartWDay,EndDate,EndWDay,StartLocLat,StartLocLon,...,MultipleCorridors,EndpointType,TripMeanSpeedKph,TripMaxSpeedKph,TripDistanceMeters,MovementType,OriginCbg,DestCbg,count,freq
565,617240f60d0baedeb0015d0f58dc85b9,cae8072aaf978457bcef17c10800d5cd,0e65972dce68dad4d52d063967f0a705,1,2017-02-11T04:22:26.000Z,6,2017-02-11T04:33:32.000Z,6,52.5142,13.4196,...,NaN,0,8.669489,60.987604,1603.855532,1,NaN,NaN,NaN,2
747,828617ef295afd8c1d5ecacd9612623f,7533e38b7971f5e07637738af5ca7656,2a79ea27c279e471f4d180b08d62b00a,1,2017-02-11T15:46:54.000Z,6,2017-02-11T16:37:33.000Z,6,52.3996,13.0478,...,NaN,3,22.541569,62.924370,19028.841320,1,NaN,NaN,NaN,2
333,381bfd824bdd6eaa2eb7c7ee81341ef4,7533e38b7971f5e07637738af5ca7656,2a79ea27c279e471f4d180b08d62b00a,1,2017-02-10T18:33:41.000Z,5,2017-02-10T19:04:18.000Z,5,52.5540,13.2923,...,NaN,3,64.723364,111.728991,33026.894502,1,NaN,NaN,NaN,2
179,1daf9ea8c82fd532d8368458ff251753,cae8072aaf978457bcef17c10800d5cd,0e65972dce68dad4d52d063967f0a705,1,2017-02-10T15:46:37.000Z,5,2017-02-10T16:57:29.000Z,5,52.2278,11.7568,...,NaN,1,105.986060,167.640970,125181.312974,1,NaN,NaN,NaN,2
666,7426be66f6b6d4e1d29faecb2a7726aa,505eadbb631f23f4e5055223bbe2cd99,52720e003547c70561bf5e03b95aa99f,1,2017-02-10T12:54:33.560Z,5,2017-02-10T13:37:20.483Z,5,52.5137,13.3312,...,NaN,0,32.163883,63.994969,22933.947160,1,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
718,7be6082f8dd97b13fa38c81a4100fb6d,7d33efc20d590cd077e3690eec71b555,00ec53c4682d36f5c4359f4ae7bd7ba1,1,2017-02-09T09:04:39.000Z,4,2017-02-09T09:11:57.000Z,4,52.4860,13.3614,...,NaN,0,22.677959,44.420703,2759.151620,1,NaN,NaN,NaN,1
717,7b971aaee83c2076a29ff7d3ebe56333,9c446ca327e854cc444f80d8fb9a7468,52720e003547c70561bf5e03b95aa99f,1,2017-02-09T07:27:31.737Z,4,2017-02-09T08:19:34.573Z,4,52.5319,13.3915,...,NaN,0,33.334722,107.077171,28916.353016,1,NaN,NaN,NaN,1
716,7b58adbab5595e8a14cfe40e6ea20a7d,355cdfe7f9d9d6ef5c9c56bda13b1f76,52720e003547c70561bf5e03b95aa99f,1,2017-02-09T17:32:46.933Z,4,2017-02-09T17:49:59.453Z,4,52.4725,13.3241,...,NaN,0,16.739477,33.417394,4801.067876,1,NaN,NaN,NaN,1
689,7782c381e079759b049eaaeb3e448672,649b10ff1ae391d17ec2c262cc20a3df,94f6d7e04a4d452035300f18b984988c,1,2017-02-10T15:51:07.000Z,5,2017-02-10T15:59:40.000Z,5,52.5000,13.4358,...,NaN,0,15.017223,18.854731,2139.954321,1,NaN,NaN,NaN,1
